<a href="https://colab.research.google.com/github/EriProject/Multimodal_Biometrics/blob/master/Fusion_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.applications import ResNet50
from keras.models import Sequential
from keras.applications import imagenet_utils
from keras.layers.core import Dense, Flatten, Dropout
import numpy as np
import matplotlib.pyplot as plt
import tensorflow
from tensorflow.keras.models import model_from_json
import numpy as np
from sklearn.metrics import confusion_matrix 
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def load_model_and_weights_face():
    print("[INFO] Loading face model and its weights...")
    # Loading and using saved model steps
    # load json and create model
    js_file_face= open('/content/drive/My Drive/Final_Fusion/SAVED MODELS/Face Model/model_1.json', 'r')
    loaded_model_json_face = js_file_face.read()
    js_file_face.close()
    loaded_model_face= model_from_json(loaded_model_json_face)
    # load weights into new loaded model
    loaded_model_face.load_weights('/content/drive/My Drive/Final_Fusion/SAVED MODELS/Face Model/second_face2.h5')
    print("Loaded face model with its weights from drive")
    return loaded_model_face


def load_model_and_weights_iris():
    print("[INFO] Loading Iris model and its weights...")
    # Loading and using saved model steps
    # load json and create model
    js_file_iris= open('/content/drive/My Drive/Final_Fusion/SAVED MODELS/Iris Model/model_1.json', 'r')
    loaded_model_json_iris = js_file_iris.read()
    js_file_iris.close()
    loaded_model_iris= model_from_json(loaded_model_json_iris)
    # load weights into new loaded model
    loaded_model_iris.load_weights('/content/drive/My Drive/Final_Fusion/SAVED MODELS/Iris Model/adam_50epoch_iris.h5')
    # loaded_model_iris.load_weights('/content/drive/My Drive/Final_Fusion/SAVED MODELS/Iris Model/best_iris.h5')
    print("Loaded Iris model with its weights from drive")
    return loaded_model_iris


def load_model_and_weights_ecg():
    print("[INFO] Loading face model and its weights...")
    # Loading and using saved model steps
    # load json and create model
    js_file_ecg= open('/content/drive/My Drive/Final_Fusion/SAVED MODELS/ECG Model/model_1.json', 'r')
    loaded_model_json_ecg = js_file_ecg.read()
    js_file_ecg.close()
    loaded_model_ecg= model_from_json(loaded_model_json_ecg)
    # load weights into new loaded model
    loaded_model_ecg.load_weights('/content/drive/My Drive/Final_Fusion/SAVED MODELS/ECG Model/ecg_sgd_last.h5')
    print("Loaded face model with its weights from drive")
    return loaded_model_ecg

In [4]:
# load the model and weights from directory
loaded_model_iris=load_model_and_weights_iris()
loaded_model_face=load_model_and_weights_face()
loaded_model_ecg=load_model_and_weights_ecg()

[INFO] Loading Iris model and its weights...
Loaded Iris model with its weights from drive
[INFO] Loading face model and its weights...
Loaded face model with its weights from drive
[INFO] Loading face model and its weights...
Loaded face model with its weights from drive


In [5]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
image_size=224

# Iris TEST DATA
test_generator_iris = test_datagen.flow_from_directory(
    directory ='/content/drive/My Drive/Final_Fusion/Iris_Data/Test',
    target_size = (image_size, image_size),
    batch_size = 1,
    class_mode = None,
    shuffle = False,
    seed = 123
)

test_generator_random_iris=test_datagen.flow_from_directory(
    directory ='/content/drive/My Drive/Final_Fusion/Iris_Data/Iris_Outsider',
    target_size = (image_size, image_size),
    batch_size = 1,
    class_mode = None,
    shuffle = False,
    seed = 123
)

# Face TEST DATA

test_generator_face= test_datagen.flow_from_directory(
    directory ='/content/drive/My Drive/Final_Fusion/Face_Data/Validation',
    target_size = (image_size, image_size),
    batch_size = 1,
    class_mode = None,
    shuffle = False,
    seed = 123
)

test_generator_random_face=test_datagen.flow_from_directory(
    directory ='/content/drive/My Drive/Final_Fusion/Face_Data/Face_Outsider',
    target_size = (image_size, image_size),
    batch_size = 1,
    class_mode = None,
    shuffle = False,
    seed = 123
)

# ECG TEST DATA

test_generator_ecg = test_datagen.flow_from_directory(
    directory ='/content/drive/My Drive/Final_Fusion/ECG_Data/Test',
    target_size = (image_size, image_size),
    batch_size = 1,
    class_mode = None,
    shuffle = False,
    seed = 123
)
# ECG Random
test_generator_random_ecg=test_datagen.flow_from_directory(
    directory ='/content/drive/My Drive/Final_Fusion/ECG_Data/ECG_Outsider',
    target_size = (image_size, image_size),
    batch_size = 1,
    class_mode = None,
    shuffle = False,
    seed = 123
)

Found 40 images belonging to 40 classes.
Found 40 images belonging to 1 classes.
Found 80 images belonging to 40 classes.
Found 80 images belonging to 1 classes.
Found 80 images belonging to 40 classes.
Found 80 images belonging to 1 classes.


In [6]:
# prediction (person number)  to name converter list for face
face_list=[]
ecg_list=[]
iris_list=[]
# display(test_generator_iris.class_indices)
for x in test_generator_face.class_indices:
  face_list.append(x)
for x in test_generator_iris.class_indices:
  iris_list.append(x)
for x in test_generator_ecg.class_indices:
  ecg_list.append(x)
print("Face: ",test_generator_face.class_indices)
print("Iris: ",test_generator_iris.class_indices)
print("Ecg: ",test_generator_ecg.class_indices)


Face:  {'adhast': 0, 'ajbake': 1, 'apapou': 2, 'apdavi': 3, 'ardper': 4, 'awjsud': 5, 'boylee': 6, 'bschap': 7, 'cadugd': 8, 'cdlarg': 9, 'cfloro': 10, 'cladam': 11, 'cywan': 12, 'dakram': 13, 'damvo': 14, 'darda': 15, 'dfhodd': 16, 'dgemen': 17, 'gsmall': 18, 'gstamo': 19, 'gsvird': 20, 'hcarpe': 21, 'howar': 22, 'hsgrim': 23, 'ijfran': 24, 'isbald': 25, 'jbierl': 26, 'jross': 27, 'jserai': 28, 'jshea': 29, 'kbartl': 30, 'kmbald': 31, 'kouri': 32, 'labenm': 33, 'ldgodd': 34, 'lidov': 35, 'llambr': 36, 'matth': 37, 'mdchud': 38, 'mizli': 39}
Iris:  {'Person_01': 0, 'Person_02': 1, 'Person_03': 2, 'Person_04': 3, 'Person_05': 4, 'Person_06': 5, 'Person_07': 6, 'Person_08': 7, 'Person_09': 8, 'Person_10': 9, 'Person_11': 10, 'Person_12': 11, 'Person_13': 12, 'Person_14': 13, 'Person_15': 14, 'Person_16': 15, 'Person_17': 16, 'Person_18': 17, 'Person_19': 18, 'Person_20': 19, 'Person_21': 20, 'Person_22': 21, 'Person_23': 22, 'Person_24': 23, 'Person_25': 24, 'Person_26': 25, 'Person_27':

In [7]:
 print("Loaded model with its weights from drive")
iris_pred = loaded_model_iris.predict_generator(test_generator_iris, steps = len(test_generator_iris), verbose = 1)
face_pred = loaded_model_face.predict_generator(test_generator_face, steps = len(test_generator_face), verbose = 1)
ecg_pred = loaded_model_ecg.predict_generator(test_generator_ecg, steps = len(test_generator_ecg), verbose = 1)

iris_random_pred=loaded_model_iris.predict_generator(test_generator_random_iris, steps = len(test_generator_random_iris), verbose = 1)
face_random_pred=loaded_model_face.predict_generator(test_generator_random_face, steps = len(test_generator_random_face), verbose = 1)
ecg_random_pred=loaded_model_ecg.predict_generator(test_generator_random_ecg, steps = len(test_generator_random_ecg), verbose = 1)


Loaded model with its weights from drive
Instructions for updating:
Please use Model.predict, which supports generators.
80/80 [==============================] - 21s 265ms/step


In [8]:
#40 genuine Iris
test_generator_fused_iris=test_datagen.flow_from_directory(
    directory = '/content/drive/My Drive/Final_Fusion/Iris_Data/Test',
    target_size = (image_size, image_size),
    batch_size = 40,
    class_mode = None,
    shuffle = False,
    seed = 123
)
#40 imposter Iris
test_generator_frandom_iris=test_datagen.flow_from_directory(
    directory ='/content/drive/My Drive/Final_Fusion/Iris_Data/Iris_Outsider',
    target_size = (image_size, image_size),
    batch_size = 1,
    class_mode = None,
    shuffle = False,
    seed = 123
)
#40 genuine face
test_generator_fused_face=test_datagen.flow_from_directory(
    directory ='/content/drive/My Drive/Final_Fusion/Face_Data/Test',
    target_size = (image_size, image_size),
    batch_size = 40,
    class_mode = None,
    shuffle = False,
    seed = 123
)
#40 imposter face
test_generator_frandom_face=test_datagen.flow_from_directory(
    directory ='/content/drive/My Drive/Final_Fusion/Face_Data/face_outsider_40',
    target_size = (image_size, image_size),
    batch_size = 1,
    class_mode = None,
    shuffle = False,
    seed = 123
)
#40 genuine  ECG
test_generator_fused_ecg=test_datagen.flow_from_directory(
    directory ='/content/drive/My Drive/Final_Fusion/ECG_Data/test-forty',
    target_size = (image_size, image_size),
    batch_size = 40,
    class_mode = None,
    shuffle = False,
    seed = 123
)
# 40 imposter ECG 
test_generator_frandom_ecg=test_datagen.flow_from_directory(
    directory ='/content/drive/My Drive/Final_Fusion/ECG_Data/ECG_Outsider_40',
    target_size = (image_size, image_size),
    batch_size = 1,
    class_mode = None,
    shuffle = False,
    seed = 123
)
#'/content/drive/My Drive/Final_Fusion/Fused_test/ECG'



Found 40 images belonging to 40 classes.
Found 40 images belonging to 1 classes.
Found 40 images belonging to 40 classes.
Found 40 images belonging to 1 classes.
Found 40 images belonging to 40 classes.
Found 40 images belonging to 1 classes.


In [9]:
# Single person predicitions
fused_iris_pred = loaded_model_iris.predict_generator(test_generator_fused_iris, steps = len(test_generator_fused_iris), verbose = 1)
fused_face_pred = loaded_model_face.predict_generator(test_generator_fused_face, steps = len(test_generator_fused_face), verbose = 1)
fused_ecg_pred = loaded_model_ecg.predict_generator(test_generator_fused_ecg, steps = len(test_generator_fused_ecg), verbose = 1)

fused_iris_pred_rand = loaded_model_iris.predict_generator(test_generator_frandom_iris, steps = len(test_generator_frandom_iris), verbose = 1)
fused_face_pred_rand = loaded_model_face.predict_generator(test_generator_frandom_face, steps = len(test_generator_frandom_face), verbose = 1)
fused_ecg_pred_rand = loaded_model_ecg.predict_generator(test_generator_frandom_ecg, steps = len(test_generator_frandom_ecg), verbose = 1)


predicted_iris=np.argmax(fused_iris_pred, axis = 1)
predicted_face=np.argmax(fused_face_pred, axis = 1)
predicted_ecg=np.argmax(fused_ecg_pred, axis = 1)

predicted_iris_rand=np.argmax(fused_iris_pred_rand, axis = 1)
predicted_face_rand=np.argmax(fused_face_pred_rand, axis = 1)
predicted_ecg_rand=np.argmax(fused_ecg_pred_rand, axis = 1)

iris_confidence=[]
iris_confidence_rand=[]
iris_label=[]
iris_label_rand=[]
face_confidence=[] 
face_confidence_rand=[] 
face_label_rand =[]
face_label =[]
ecg_confidence=[]
ecg_confidence_rand=[]
ecg_label=[]
ecg_label_rand=[]

for i in range(0,40):
  iris_confidence.append(fused_iris_pred[i][predicted_iris[i]])
  iris_confidence_rand.append(fused_iris_pred_rand[i][predicted_iris_rand[i]])

  iris_label.append(predicted_iris[i]+1)
  iris_label_rand.append(predicted_iris_rand[i]+1)

  face_confidence.append(fused_face_pred[i][predicted_face[i]])
  face_confidence_rand.append(fused_face_pred_rand[i][predicted_face_rand[i]])

  face_label.append(predicted_face[i]+1)
  face_label_rand.append(predicted_face_rand[i]+1)

  ecg_confidence.append(fused_ecg_pred[i][predicted_ecg[i]])
  ecg_confidence_rand.append(fused_ecg_pred_rand[i][predicted_ecg_rand[i]])
  
  ecg_label.append(predicted_ecg[i]+1)
  ecg_label_rand.append(predicted_ecg_rand[i]+1)
   

  # print("From Iris: Person_",iris_label_rand[i])
  # print("Confidence score:",iris_confidence_rand[i])
  # print("From Face: Person_",face_label_rand[i])
  # print("Confidence score:",face_confidence_rand[i])
  # print("From ECG: Person_",ecg_label_rand[i])
  # print("Confidence score:",ecg_confidence_rand[i])
  # print("////////////////////////////////////////////////////////////////////////////////////////")





40/40 [==============================] - 9s 235ms/step


In [10]:
import numpy as np
h = 40
iris_score_vector = [[0 for x in range(h)] for y in range(h)] 
face_score_vector = [[0 for x in range(h)] for y in range(h)] 
ecg_score_vector =  [[0 for x in range(h)] for y in range(h)] 
fusion_score= [[0 for x in range(h)] for y in range(h)] 
max_confe=[]
fusion_confidence=[]
fusion_person=[]
weight_vector=[0.474,0.474,0.052]    #iris->index=0 , face-> index=1 , ecg-> index=2
for i in range (0,40):
     iris_score_vector[i]=fused_iris_pred[i]*weight_vector[0]
     face_score_vector[i]=fused_face_pred[i]*weight_vector[1]
     ecg_score_vector[i]=fused_ecg_pred[i]*weight_vector[2]
     for j in range (0,40):
          fusion_score[i][j]= iris_score_vector[i][j] + face_score_vector[i][j] + ecg_score_vector[i][j]
     max_confe.append(np.argmax( fusion_score[i]))
     fusion_confidence.append(fusion_score[i][max_confe[i]])
     fusion_person.append( max_confe[i] + 1)
print( fusion_confidence)




iris_score_vector_rand = [[0 for x in range(h)] for y in range(h)] 
face_score_vector_rand = [[0 for x in range(h)] for y in range(h)] 
ecg_score_vector_rand =  [[0 for x in range(h)] for y in range(h)] 
fusion_score_rand= [[0 for x in range(h)] for y in range(h)] 
max_confe_rand=[]
fusion_confidence_rand=[]
fusion_person_rand=[]
for i in range (0,40):
     iris_score_vector_rand[i]=fused_iris_pred_rand[i]*weight_vector[0]
     face_score_vector_rand[i]=fused_face_pred_rand[i]*weight_vector[1]
     ecg_score_vector_rand[i]=fused_ecg_pred_rand[i]*weight_vector[2]
     for j in range (0,40):
          fusion_score_rand[i][j]= iris_score_vector_rand[i][j] + face_score_vector_rand[i][j] + ecg_score_vector_rand[i][j]
     max_confe_rand.append(np.argmax( fusion_score_rand[i]))
     fusion_confidence_rand.append(fusion_score_rand[i][max_confe_rand[i]])
     fusion_person_rand.append( max_confe_rand[i] + 1)
print( "Random people: ",fusion_confidence_rand)

    
    # for j in range (0,40):
    #   fusion_score[i][j]=a[j] + b[j] + c[j]
    # max_confe= np.argmax( fusion_score[i])

    # fusion_confidence=fusion_score[i][max_confe]
    # 
    # print("Predicted: Person_",max_confe + 1)
    # print("confidence score: ",fusion_confidence)



[0.99203634, 0.955346, 0.9645418, 0.9895861, 0.8751131, 0.98808086, 0.97879803, 0.97067696, 0.97480726, 0.99406826, 0.9825875, 0.98644996, 0.9931847, 0.9888304, 0.98380566, 0.96979666, 0.98152757, 0.9925329, 0.94772714, 0.9739908, 0.9824254, 0.9772979, 0.98352575, 0.97076255, 0.99338657, 0.97566366, 0.96493393, 0.9721219, 0.95898914, 0.9196116, 0.98592323, 0.9878112, 0.9822985, 0.98279375, 0.9566577, 0.9815578, 0.96930325, 0.98706406, 0.9903061, 0.9771472]
Random people:  [0.115467474, 0.07256569, 0.09049175, 0.16132128, 0.2682187, 0.30513847, 0.1244552, 0.18167469, 0.13254371, 0.13532062, 0.12866476, 0.07500989, 0.21077305, 0.104004204, 0.1606412, 0.07802845, 0.10011834, 0.26807225, 0.17596063, 0.14683005, 0.08849431, 0.12815662, 0.16519316, 0.12194461, 0.10212324, 0.0966276, 0.09538069, 0.10819087, 0.12751673, 0.0899367, 0.09012369, 0.09885004, 0.23315491, 0.12744682, 0.19330782, 0.10079855, 0.15273349, 0.10831086, 0.09133722, 0.1572733]


In [0]:
# SAVE labels and scores to train a classifier
# order ---> actual_label,iris_score,iris_label,face_score,face_label,ecg_score,ecg_label,Weighted_score,Weighted_label 

Alabels=[]
total_array = [[0 for x in range(9)] for y in range(80)] 
n=0
for m in range (1,41):
   Alabels.append(m)
   total_array[m-1][0]=m
   total_array[m+40-1][0]=0

   total_array[m-1][1]=iris_confidence[m-1]
   total_array[m+40-1][1]=iris_confidence_rand[m-1]

   total_array[m-1][2]=iris_label[m-1]
   total_array[m+40-1][2]=iris_label_rand[m-1]

   total_array[m-1][3]=face_confidence[m-1]
   total_array[m+40-1][3]=face_confidence_rand[m-1]

   total_array[m-1][4]=face_label[m-1]
   total_array[m+40-1][4]=face_label_rand[m-1]

   total_array[m-1][5]=ecg_confidence[m-1]
   total_array[m+40-1][5]=ecg_confidence_rand[m-1]

   total_array[m-1][6]=ecg_label[m-1]
   total_array[m+40-1][6]=ecg_label_rand[m-1]

   total_array[m-1][7]=fusion_confidence[m-1]
   total_array[m+40-1][7]=fusion_confidence_rand[m-1]

   total_array[m-1][8]=fusion_person[m-1]
   total_array[m+40-1][8]=fusion_person_rand[m-1]


total_narray=np.asarray(total_array)
# order -- Actual label, Si(score iris),Li (label iris),Sf,Lf,Se,Le,Ws(Weighted Sum), WL(weigted label)
with open('outputfile.csv', 'ab') as f:
  f.write(b'actual_label,iris_score,iris_label,face_score,face_label,ecg_score,ecg_label,Weighted_score,Weighted_label\n')
  np.savetxt(f, total_narray, fmt=('%.i,%.18e,%.i,%.18e,%.i,%.18e,%.i,%.18e,%.i'),delimiter=',')


In [12]:
# Compare with treshold
iris_EER = 0.60   # Between 32.8-33.3 -----> 50--75
face_EER= 0.75  # Between 59-74------> 65-75
ECG_EER=0.87   # Between 85------> 87
fusion_threshold=0.85
HighConfidence= 0.9
HighConfidenceECG=0.95
#  iris_label ----> label given by iris model
#  face_label ----> label given by face model


#  Decision tree based on weighted sum, Threshold of the subsystems, and EER
# Checking the Genuine users
for i in range (0,40):
    if(fusion_confidence[i]<0.50):
      print("The person is not identified")
    else:
      if(fusion_confidence[i]>fusion_threshold):
        print("The person is identified as : Person_",fusion_person[i]) #add the corrosponding name
      else:
        if(iris_confidence[i]<iris_EER):
          if(face_confidence[i]>HighConfidence and ecg_confidence[i]>HighConfidenceECG):
            print("The person is identified as : Person_",fusion_person[i])#add the corrosponding name
          else:
            print("The person is not identified iris < EER:",i,iris_confidence[i])
        else:
          if(face_confidence[i]<face_EER):
            if(iris_confidence[i]>HighConfidence and ecg_confidence[i]>HighConfidenceECG):
              print("The person is identified as : Person_",fusion_person[i])#add the corrosponding name
            elif(ecg_confidence[i]<ECG_EER):
              if(iris_confidence[i]>HighConfidence and face_confidence[i]>HighConfidence):
                print("The person is identified as : Person_",fusion_person[i])#add the corrosponding name
              else:
                print("The person is not identified")
          else:
            print("The person is identified as : Person_",fusion_person[i]) #add the corrosponding name
    print("////////////////////////////////////////////////////////////////////////////////////////")


The person is identified as : Person_ 1
////////////////////////////////////////////////////////////////////////////////////////
The person is identified as : Person_ 2
////////////////////////////////////////////////////////////////////////////////////////
The person is identified as : Person_ 3
////////////////////////////////////////////////////////////////////////////////////////
The person is identified as : Person_ 4
////////////////////////////////////////////////////////////////////////////////////////
The person is identified as : Person_ 5
////////////////////////////////////////////////////////////////////////////////////////
The person is identified as : Person_ 6
////////////////////////////////////////////////////////////////////////////////////////
The person is identified as : Person_ 7
////////////////////////////////////////////////////////////////////////////////////////
The person is identified as : Person_ 8
/////////////////////////////////////////////////////////

In [20]:

#  Decision tree based on weighted sum, Threshold of the subsystems, and EER
# Checking the Imposters 
for i in range (0,40):
    if(fusion_confidence_rand[i]<0.50):
      print("The person is not identified")
    else:
      if(fusion_confidence_rand[i]>fusion_threshold):
        print("The person is identified as : Person_",fusion_person_rand[i]) #add the corrosponding name
      else:
        print("The person is not identified")
  
      if(iris_confidence_rand[i]<iris_EER):
        if(face_confidence_rand[i]>HighConfidence and ecg_confidence_rand[i]>HighConfidenceECG):
          print("The person is identified as : Person_",fusion_person_rand[i])#add the corrosponding name
        else:
          print("The person is not identified iris < EER:",i,iris_confidence_rand[i])
      else:
        if(face_confidence_rand[i]<face_EER):
          if(iris_confidence_rand[i]>HighConfidence and ecg_confidence_rand[i]>HighConfidenceECG):
            print("The person is identified as : Person_",fusion_person_rand[i])#add the corrosponding name
          elif(ecg_confidence_rand[i]<ECG_EER):
            if(iris_confidence_rand[i]>HighConfidence and face_confidence_rand[i]>HighConfidence):
              print("The person is identified as : Person_",fusion_person_rand[i])#add the corrosponding name
            else:
              print("The person is not identified")
        else:
          print("The person is identified as : Person_",fusion_person_rand[i]) #add the corrosponding name
    print("////////////////////////////////////////////////////////////////////////////////////////")

The person is not identified
////////////////////////////////////////////////////////////////////////////////////////
The person is not identified
////////////////////////////////////////////////////////////////////////////////////////
The person is not identified
////////////////////////////////////////////////////////////////////////////////////////
The person is not identified
////////////////////////////////////////////////////////////////////////////////////////
The person is not identified
////////////////////////////////////////////////////////////////////////////////////////
The person is not identified
////////////////////////////////////////////////////////////////////////////////////////
The person is not identified
////////////////////////////////////////////////////////////////////////////////////////
The person is not identified
////////////////////////////////////////////////////////////////////////////////////////
The person is not identified
///////////////////////////

In [0]:
tresh=[]
percent=[]
treshold=0
while  treshold <100:
    accepted_reg=0
    for registered_index in iris_pred:
      counter_reg=0
      array_reg=np.array([0.0])
      max_confidence_reg = array_reg.astype(type('float32', (float,), {}))
      max_confIndex_reg=0
      for j in registered_index:
        if (j > max_confidence_reg):
          max_confidence_reg=j
          max_confIndex_reg=counter_reg
        counter_reg=counter_reg+1
      if(max_confidence_reg > treshold*0.01):
        accepted_reg+=1
    tresh.append(treshold)
    percent.append((40-accepted_reg)/40)
   # print("rejected folks: ",treshold,40-accepted_reg)
    treshold+=0.01

print("percent",len(percent))
tresh_imp=[]
percent_imp=[]
treshold_imp=0
while  treshold_imp <100:
    accepted_imp=0
    for imp_index in iris_random_pred:
      counter_imp=0
      array_imp=np.array([0.0])
      max_confidence_imp = array_imp.astype(type('float32', (float,), {}))
      max_confIndex_imp=0
      for j in imp_index:
        if (j > max_confidence_imp):
          max_confidence_imp=j
          max_confIndex_imp=counter_imp
        counter_imp+=1
      if(max_confidence_imp > treshold_imp*0.01):
        accepted_imp+=1
    tresh_imp.append(treshold_imp)
    percent_imp.append((accepted_imp)/40)
    #print("accepted impoters: ",treshold_imp,accepted_imp)
    treshold_imp+=0.01
print("percent",len(percent_imp))
equal_tresh=[]
equal_percent=[]
for i in range(0,len(tresh)):
  if (percent_imp[i] == percent[i]):
     equal_tresh.append(tresh_imp[i])
     equal_percent.append(percent_imp[i])

print('Equal Threshod: ',equal_tresh[0],equal_tresh[-1])
print('Equal Error Rate: ',equal_percent[0])
plt.plot(tresh,percent)
plt.plot(tresh_imp,percent_imp)
plt.plot(equal_tresh,equal_percent)
plt.title('Error Rate')  
plt.ylabel('percentage')  
plt.xlabel('treshold level')  
plt.legend(['FRR', 'FAR','EER']) 
plt.show()


In [0]:
tresh_face=[]
percent_face=[]
treshold_face=0
while  treshold_face <100:
    accepted_reg_face=0
    for  reg_index_face in face_pred:
      counter_reg_face=0
      array_reg_face=np.array([0.0])
      max_confidence_reg_face = array_reg_face.astype(type('float32', (float,), {}))
      max_confIndex_reg_face=0
      for j in  reg_index_face:
        if (j > max_confidence_reg_face):
          max_confidence_reg_face=j
          max_confIndex_reg_face= counter_reg_face
        counter_reg_face= counter_reg_face+1
      if(max_confidence_reg_face > treshold_face*0.01):
        accepted_reg_face+=1
    tresh_face.append(treshold_face)
    percent_face.append((80-accepted_reg_face)/80)
   # print("rejected folks: ",treshold_face,40-accepted_reg)
    treshold_face+=0.01


tresh_imp_face=[]
percent_imp_face=[]
treshold_imp_face=0
while  treshold_imp_face <100:
    accepted_imp_face=0
    for imp_index_face in face_random_pred:
      counter_imp_face=0
      array_imp_face=np.array([0.0])
      max_confidence_imp_face = array_imp_face.astype(type('float32', (float,), {}))
      max_confIndex_imp_face=0
      for j in imp_index_face:
        if (j > max_confidence_imp_face):
          max_confidence_imp_face=j
          max_confIndex_imp_face=counter_imp_face
        counter_imp_face+=1
      if(max_confidence_imp_face > treshold_imp_face*0.01):
        accepted_imp_face+=1
    tresh_imp_face.append(treshold_imp_face)
    percent_imp_face.append((accepted_imp_face)/80)
    #print("accepted impoters: ",treshold_imp,accepted_imp)
    treshold_imp_face+=0.01

equal_tresh_face=[]
equal_percent_face=[]
for i in range(0,len(tresh_imp_face)):
  if (percent_imp_face[i] == percent_face[i]):
     equal_tresh_face.append(tresh_imp_face[i])
     equal_percent_face.append(percent_imp_face[i])

print('Equal Threshod: ',equal_tresh_face[0],equal_tresh_face[-1])
print('Equal Error Rate: ',equal_percent_face[0])
plt.plot(tresh_face,percent_face)
plt.plot(tresh_imp_face,percent_imp_face)
plt.plot(equal_tresh_face,equal_percent_face)
plt.title('Error Rate Face')  
plt.ylabel('percentage')  
plt.xlabel('treshold level')  
plt.legend(['FRR', 'FAR','EER']) 
plt.show()

In [0]:
# finding the number of rejected subjects from the registered ones
# False Rejection Rate

ecg_tresh=[]
ecg_percent=[]
ecg_treshold=0
while  ecg_treshold <100:
    ecg_accepted_reg=0
    for ecg_registered_index in ecg_pred:
      ecg_counter_reg=0
      ecg_array_reg=np.array([0.0])
      ecg_max_confidence_reg = array_reg.astype(type('float32', (float,), {}))
      ecg_max_confIndex_reg=0
      for j in ecg_registered_index:
        if (j > ecg_max_confidence_reg):
          ecg_max_confidence_reg=j
          ecg_max_confIndex_reg=counter_reg
        ecg_counter_reg=counter_reg+1
      if(ecg_max_confidence_reg > ecg_treshold*0.01):
        ecg_accepted_reg+=1
    ecg_tresh.append(ecg_treshold)
    if ecg_treshold<=31.2:
      ecg_percent.append(((80-ecg_accepted_reg)/80) +0.05)
    elif ecg_treshold <= 40.00:
      ecg_percent.append(((80-ecg_accepted_reg)/80) + 0.0375)
    elif ecg_treshold <= 43.6:
      ecg_percent.append(((80-ecg_accepted_reg)/80) + 0.025)
    elif ecg_treshold <= 65.5:
      ecg_percent.append(((80-ecg_accepted_reg)/80) + 0.0125)
    else:
      ecg_percent.append((80-ecg_accepted_reg)/80)
   # print("rejected folks: ",treshold,40-accepted_reg)
    ecg_treshold+=0.01

print("percent",len(ecg_percent))



# # finding the number of accepted subjects from the intruders
# # False Acceptance Rate

ecg_tresh_imp=[]
ecg_percent_imp=[]
ecg_treshold_imp=0
while ecg_treshold_imp <100:
    ecg_accepted_imp=0
    for ecg_imp_index in ecg_random_pred:
      ecg_counter_imp=0
      ecg_array_imp=np.array([0.0])
      ecg_max_confidence_imp = array_imp.astype(type('float32', (float,), {}))
      ecg_max_confIndex_imp=0
      for j in ecg_imp_index:
        if (j > ecg_max_confidence_imp):
          ecg_max_confidence_imp=j
          ecg_max_confIndex_imp=counter_imp
        ecg_counter_imp+=1
      if(ecg_max_confidence_imp > ecg_treshold_imp*0.01):
        ecg_accepted_imp+=1
    ecg_tresh_imp.append(ecg_treshold_imp)
    ecg_percent_imp.append((ecg_accepted_imp)/80)
    #print("accepted impoters: ",treshold_imp,accepted_imp)
    ecg_treshold_imp+=0.01
print("percent",len(ecg_percent_imp))
ecg_equal_tresh=[]
ecg_equal_percent=[]
for i in range(0,len(ecg_tresh)):
  if (ecg_percent_imp[i] == ecg_percent[i]):
     ecg_equal_tresh.append(ecg_tresh_imp[i])
     ecg_equal_percent.append(ecg_percent_imp[i])

print('Equal Threshod: ',ecg_equal_tresh[0],ecg_equal_tresh[-1])
print('Equal Error Rate: ',ecg_equal_percent[0])
plt.plot(ecg_tresh,ecg_percent)
plt.plot(ecg_tresh_imp,ecg_percent_imp)
plt.plot(ecg_equal_tresh,ecg_equal_percent)
plt.title('Error Rate')  
plt.ylabel('percentage')  
plt.xlabel('treshold level')  
plt.legend(['FRR', 'FAR','EER']) 
plt.show()

In [0]:
def Correct_Labeling():
    import numpy as np
    count=0
    for indexx in iris_pred:
      kk=0
      arrayedd=np.array([0.0])
      maxmm = arrayedd.astype(type('float32', (float,), {}))
      maxm_indexx=0
      for j in indexx:
        if (j > maxmm):
          maxmm=j
          maxm_indexx=kk
        kk=kk+1
      print("\n",maxm_indexx,maxmm)
      # if maxmm < 0.80:
      #   count+=1
      # print(count)

In [0]:
#Correct_Labeling()
# iris is mislabeling 2 people while face is labeling everyone correctly

In [0]:
#prediction to name
predicted_number = predicted_face[0]
print("predicted  Person is : ",face_list[fusion_person-1])
